# Aufgabe-D-Schilder

## Allgemeines

Eine allgemeine Beschreibung der Laboraufgaben inklusive des Vorgehens, den Bewertungsrichtlinien und der Abgabe finden Sie  <a href="ML-allgemein.ipynb">hier</a>

## VM

1. Ping external IP (Floating IP)
1.1 Not working? --> New Floating IP
1.2 Working --> ssh ubuntu@141.72.191.72 -i privatekey.txt

## Datenquelle


* Laden Sie ihre Daten von http://141.72.190.207/ml_lab/D_schilder herunter
    * Die Daten sind geschützt. 
    * Sie müssen evtl. in einem Netzwerk der DHBW (z.B. WLAN, VPN, ...) angemeldet sein. 
        * Sie können sich auf der Webseite mit dem Benutzernamen dhbw und dem Zugangsnamen ml_2021 anmelden. 
* Die Daten sind in einem anwendungsspezifischen Format gespeichert.
    * Sie finden evtl. Informationen über die Daten in einer "README" Datei. 
    * Finden Sie keine solche Datei sind die Daten selbst erklärend. 
    
    



## Aufgabe

* die Daten 
    * enthalten mehr als 50000 Bilder von Verkehrsschilder auf deutschen Straßen
    * pro Klasse der Daten gibt es ein Verzeichnis
    * in jeder Klasse gibt eine CSV-Datei mit Annotationen der Bilder
        * Die Namen lauten GT-<ClassID>.csv
* die Bilder 
    * enthalten genau ein Verkehrszeichen, mit ca. 10% Rand um das Verkehrszeichen
    * sind im PPM Format gespeichert
    * die Größen der Bilder variieren von 15x15 Pixel bis zu 250x250 Pixel
        * die Bilder müssen aber nicht quadratisch sein
    * In den Annotationen ist auch die Bounding Box hinterlegt
* die Annotations-Datei hat den folgenden Werte
    * Dateiname, Width, Height, BB.x1, BB.y1, BB.x2, BB.y2, ClassId
        * BB steht für BoundingBox

Erstellen Sie ein tiefes neuronales Netz, dass Schilder in Bildern erkennt und die Position des erkannten Schilds als Bounding Box ausgibt! Die Lösung soll auch mehrere Schilder in einem Bild erkennen können! 


# Lösung

* Beginnen Sie hier mit Ihrer Dokumentation und Implementierung! 

In [2]:
import cv2
import pandas as pd

Funktion "drawImage" --> BB und Image wird dargestellt, mapping über Namen<br>
Einlesen von Bildern mit zugehörigen Namen

In [3]:
def read_annotations(folders=43, pre_path=".\\data\\") -> pd.DataFrame:
    
    df_annotations = pd.DataFrame()
    for i in range(0, folders):
        if(i<10):
            path = pre_path + str(i) + "\\GT-0000" + str(i) +".csv"
        else:
            path = pre_path + str(i) + "\\GT-000" + str(i) +".csv"
        
        df_annotations = df_annotations.append(pd.read_csv(path, sep=";"))
    df_annotations.reset_index(inplace=True)
    df_annotations.drop("index", axis=1, inplace=True)
    return df_annotations

In [4]:
df_annotations = read_annotations()
df_annotations

,Filename,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId
0,00000_00000.ppm,29,30,5,6,24,25,0
1,00000_00001.ppm,30,30,5,5,25,25,0
2,00000_00002.ppm,30,30,5,5,25,25,0
3,00000_00003.ppm,31,31,5,5,26,26,0
4,00000_00004.ppm,30,32,5,6,25,26,0
...,...,...,...,...,...,...,...,...
39204,00007_00025.ppm,52,56,5,6,47,51,42
39205,00007_00026.ppm,56,58,5,5,51,53,42
39206,00007_00027.ppm,58,62,5,6,53,57,42
39207,00007_00028.ppm,63,69,5,7,58,63,42


In [5]:
def read_files(folders=43, pre_path=".\\data\\") -> pd.DataFrame:
 
    df_images = pd.DataFrame()
    for i in range(0, folders):
        if(i<10):
            path = pre_path + str(i) + "\\GT-0000" + str(i) +".csv"
        else:
            path = pre_path + str(i) + "\\GT-000" + str(i) +".csv"
        
        df_tmp = pd.read_csv(path, sep=";")
        filenames = df_tmp["Filename"]
        
        df_files_and_folder = pd.DataFrame() 
        df_files_and_folder["Filename"] = filenames
        df_files_and_folder["Folder"] = ".\\data\\" + str(i) + "\\"
        df_images = df_images.append(df_files_and_folder)
        
    df_images.reset_index(inplace=True)
    df_images.drop("index", axis=1, inplace=True)
    return df_images

In [6]:
df_images = read_files()
df_images

,Filename,Folder
0,00000_00000.ppm,.\data\0\
1,00000_00001.ppm,.\data\0\
2,00000_00002.ppm,.\data\0\
3,00000_00003.ppm,.\data\0\
4,00000_00004.ppm,.\data\0\
...,...,...
39204,00007_00025.ppm,.\data\42\
39205,00007_00026.ppm,.\data\42\
39206,00007_00027.ppm,.\data\42\
39207,00007_00028.ppm,.\data\42\


In [8]:
print(len(df_images))
for i in range(20000, len(df_images)-1):
    img = cv2.imread(df_images["Folder"].iloc[i] + df_images["Filename"].iloc[i])
    
    print(df_annotations.index[df_annotations["Filename"] == df_images["Filename"].iloc[i]])
    break
    start_point = (df_annotations["Roi.X1"].iloc[i], df_annotations["Roi.Y1"].iloc[i])
    end_point = (df_annotations["Roi.X2"].iloc[i], df_annotations["Roi.Y2"].iloc[i])
    img = cv2.rectangle(img, start_point, end_point, (255,0,0), 1)
    
    img = cv2.resize(img, (200, 200))
    #cv2.imshow("Image", img)
    #k = cv2.waitKey(0)
    cv2.destroyAllWindows()

39209
Int64Index([2210, 4430, 16670, 20000, 22100, 37999], dtype='int64')


In [ ]:
data_path = ".\\data\\"

img = cv2.imread(data_path + "0\\00000_00001.ppm")
img2 = cv2.imread(data_path + "0\\00000_00002.ppm")

start_point = (5, 6)
end_point = (24, 25)

img = cv2.rectangle(img, start_point, end_point, (255,0,0), 2)

print(img.shape)

In [ ]:
img = cv2.resize(img, (400, 400))
cv2.imshow("Image", img)
k = cv2.waitKey(0)
cv2.destroyAllWindows()